In [2]:
'''
    1-D Expected Improvement
    Taken from Professor Varun Notebook
'''
from scipy.stats import norm
from scipy.optimize import minimize, fmin
import GPy
import numpy as np

def expected_improvement(X, X_sample, Y_sample, gpr, xi=0.01, mpi=False):
    '''
    Computes the EI at points X based on existing samples X_sample
    and Y_sample using a Gaussian process surrogate model.
    
    Args:
        X: Points at which EI shall be computed (m x d).
        X_sample: Sample locations (n x d).
        Y_sample: Sample values (n x 1).
        gpr: A GaussianProcessRegressor fitted to samples.
        xi: Exploitation-exploration trade-off parameter.
    
    Returns:
        Expected improvements at points X.
    '''
    mu, sigma = gpr.predict(X)
    mu_sample = gpr.predict(X_sample)

    sigma = sigma.reshape(-1, 1)
    
    # Needed for noise-based model,
    # otherwise use np.max(Y_sample).
    mu_sample_opt = np.max(mu_sample)

    with np.errstate(divide='warn'):
        imp = mu - mu_sample_opt - xi
        Z = imp / sigma
        ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z)
        ei[sigma == 0.0] = 0.0
    if(mpi):
        return norm.cdf(Z)
    return ei.flatten()

def propose_location(acquisition, X_sample, Y_sample, gpr, bounds, n_restarts=25):
    '''
    Proposes the next sampling point by optimizing the acquisition function.
    
    Args:
        acquisition: Acquisition function.
        X_sample: Sample locations (n x d).
        Y_sample: Sample values (n x 1).
        gpr: A GaussianProcessRegressor fitted to samples.

    Returns:
        Location of the acquisition function maximum.
    '''
    dim = X_sample.shape[1]
    min_val = 1
    min_x = None
    
    def min_obj(X):
        # Minimization objective is the negative acquisition function
        return -acquisition(X.reshape(-1, dim), X_sample, Y_sample, gpr)
    
    # Find the best optimum by starting from n_restart different random points.
    for x0 in np.random.uniform(bounds[:, 0], bounds[:, 1], size=(n_restarts, dim)):
        res = minimize(min_obj, x0=x0, bounds=bounds, method='L-BFGS-B')        
        if res.fun < min_val:
            min_val = res.fun[0]
            min_x = res.x           
            
    return min_x.reshape(-1, 1)

def min_obj(X):
    # Minimization objective is the negative acquisition function
    return -expected_improvement(X.reshape(-1, dim), X_sample, Y_sample, gpr)

In [ ]:
'''
    2-D Expected Improvement
'''
def expected_improvement_2d(X, X_sample, Y_sample, gpr, xi=0.01, mpi=False,plot=False):
    '''
    Computes the EI at points X based on existing samples X_sample
    and Y_sample using a Gaussian process surrogate model.
    
    Args:
        X: Points at which EI shall be computed (m x d).
        X_sample: Sample locations (n x d).
        Y_sample: Sample values (n x 1).
        gpr: A GaussianProcessRegressor fitted to samples.
        xi: Exploitation-exploration trade-off parameter.
    
    Returns:
        Expected improvements at points X.
    '''
    #print(plot)
    if plot:
        x1=X[:,0]
        x2=X[:,1]
        X,Y=grid=np.meshgrid(x1,x2)
        X=np.vstack([X.flatten(),Y.flatten()]).T
        #print(X.shape)
    mu, sigma = gpr.predict(X)
    mu_sample = gpr.predict(X_sample)

    sigma = sigma.reshape(-1, 1)
    
    # Needed for noise-based model,
    # otherwise use np.max(Y_sample).
    mu_sample_opt = np.max(mu_sample)

    with np.errstate(divide='warn'):
        imp = mu - mu_sample_opt - xi
        Z = imp / sigma
        ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z)
        ei[sigma == 0.0] = 0.0
    if(mpi):
        return norm.cdf(Z)
    return ei.flatten()

def propose_location_2d(acquisition, X_sample, Y_sample, gpr, bounds, n_restarts=25):
    '''
    Proposes the next sampling point by optimizing the acquisition function.
    
    Args:
        acquisition: Acquisition function.
        X_sample: Sample locations (n x d).
        Y_sample: Sample values (n x 1).
        gpr: A GaussianProcessRegressor fitted to samples.

    Returns:
        Location of the acquisition function maximum.
    '''
    dim = X_sample.shape[1]
    min_val = 1
    min_x = None
    
    def min_obj(X):
        # Minimization objective is the negative acquisition function
        return -acquisition(X.reshape(-1, dim), X_sample, Y_sample, gpr)
    # Find the best optimum by starting from n_restart different random points.
    for (x0,x1) in np.random.uniform(bounds[:, 0], bounds[:, 1], size=(n_restarts, dim)):
    #for (x0,x1) in np.random.uniform(bounds[0], bounds[1], size=(n_restarts, dim)):
        #res = minimize(min_obj, x0=[x0,x1], bounds=np.vstack((bounds, bounds)), method='L-BFGS-B')
        #res = minimize(min_obj, x0=[x0,x1], bounds=((-1, 1), (-1, 1)), method='TNC')
        res = minimize(min_obj, x0=[x0,x1], \
                       bounds=bounds, method='L-BFGS-B')
        if res.fun < min_val:
            min_val = res.fun[0]
            min_x = res.x           
            
    return min_x.reshape(-1, 1)

def min_obj(X):
    # Minimization objective is the negative acquisition function
    return -expected_improvement_2d(X.reshape(-1, dim), X_sample, Y_sample, gpr)

In [ ]:
'''
    Sampling a Gaussian from the posterior mu, var
    fit from the augmented data fit posterior 
'''
def sample_gaussian(mu,var):
    sample=[]
    for i,mean in enumerate(mu):
        #sample.append(np.random.multivariate_normal(np.asarray(mean),np.asarray(var[i]).reshape(-1,1))[0])
        sam=np.random.multivariate_normal(np.asarray(mean),np.asarray(var[i]).reshape(-1,1))[0]
        if var[i]>50:
            sam = mean + (var[i]/2)*(np.random.choice(np.array([-1,1])))*np.random.sample()
        sample.append(sam)
    return np.array(sample)


def thompson_sampling(n_iter, X, X_init, Y_init, f, kernel, \
             mf=None, lr=0.4,plotPlots=False):
    '''
        Thompson Sampling
    '''
    X_sample = X_init
    Y_sample = Y_init 
    for i in range(n_iter):
        if(mf):
            gpr = GPy.models.GPRegression(X_sample,Y_sample,kernel=kernel, \
                                          noise_var=.3, mean_function=mf)
        else:
            gpr = GPy.models.GPRegression(X_sample,Y_sample,kernel=kernel)

        mu, var = gpr.predict(X)
        cov = gpr.kern.K(X)
        thompson_sample = sample_gaussian(mu, var)
        Y_eval_index = np.argmax(thompson_sample)
        X_next = X[Y_eval_index]
        Y_next = f(gamma=X_next, live=False, lr=lr, best_samples=Y_p.flatten()) + np.random.rand() * -100 * (np.random.choice(np.array([-1,1])))
        
        if(plotPlots):
            plt.subplot(n_iter, 1, i + 1)
            plt.fill_between(X.ravel(), (mu - var).ravel(), (mu + var).ravel(), alpha=0.2)
            plt.plot(X.ravel(), mu.ravel(), label='Mean')
            plt.plot(X.ravel(), thompson_sample, lw=2, ls='--', label='Thompson Sample')
            plt.plot(X_sample, Y_sample, 'kx', mew=1.8, label='Acquisition Fn - Next Sample')
            #plt.xlabel('Gamma')
            #plt.ylabel('Taxi Reward')
            plt.axvline(x=X_next, ls='--', c='k', lw=1, label='Next sampling location')
            plt.legend(bbox_to_anchor=(1,1), loc="upper left")
        
        X_sample = np.vstack((X_sample, X_next))
        Y_sample = np.vstack((Y_sample, Y_next))
    return X_sample, Y_sample

In [ ]:
'''
    Plot methods
    * 1D case - Professor Varun's plot function from Notebooks
    * 2D case
'''
import matplotlib.pyplot as plt

def plot_contour(gpr, X, X_sample, Y_sample, X_next=np.array([]), noisy_label='Noisy', show_legend=False):
    #print('x',X[0].shape)
    x1,x2=X
    X,Y=grid=np.meshgrid(x1,x2)
    x_test=np.vstack([X.flatten(),Y.flatten()]).T
    #print('x_test',x_test.shape)
    mu,var=gpr.predict(x_test)
    means=mu
    plot_means=means.reshape(X[0].shape[0],X[0].shape[0])
    #print('x_sample ',X_sample.shape, x_test.shape)
    plt.plot(X_sample[:, 0], X_sample[:, 1], 'kx', mew=3, label='Noisy samples')
    if(X_next.size > 0):
        plt.plot(X_next.T[:, 0], X_next.T[:, 1], 'co', mew=3, label='Acquisition Fn - Next Sample')
    #CS = plt.contour(X, Y, plot_means,levels=200, linewidths=0.5, colors='k')
    cntr1 = plt.contourf(X, Y, plot_means, levels=700, cmap="RdBu_r")
    #plt.clabel(CS, inline=1, fontsize=10)
    if(show_legend):
        plt.legend()

    
def plot_contour_var(gpr, X, X_sample, Y_sample,  X_next=np.array([]), noisy_label='Noisy', showLegend=False):
    x1,x2=X
    X,Y=grid=np.meshgrid(x1,x2)
    x_test=np.vstack([X.flatten(),Y.flatten()]).T
    mu,var=gpr.predict(x_test)
    means=var
    plot_means=means.reshape(X[0].shape[0],X[0].shape[0])
    #print('x_sample ',X_sample.shape, x_test.shape)
    plt.plot(X_sample[:, 0], X_sample[:, 1], 'kx', mew=1, label='Samples')
    #CS = plt.contour(X, Y, plot_means,levels=200, linewidths=0.5, colors='k')
    cntr1 = plt.contourf(X, Y, plot_means, levels=700, cmap="RdBu_r")
    if(X_next.size > 0):
        plt.plot(X_next.T[:, 0], X_next.T[:, 1], 'co', mew=3, label='Acquisition Fn - Next Sample')
    #plt.clabel(CS, inline=1, fontsize=10)
    if(showLegend):
        plt.legend()
        #plt.colorbar(cntr1)#, ax=ax)
    
def plot_app(gpr, X, X_sample, Y_sample, X_next=None, noisy_label='Noisy', show_legend=False):
    mu, std = gpr.predict(X)
    #print(X.ravel())
    #print(mu.ravel())
    #print(std.ravel())
    plt.fill_between(X.ravel(), 
                     mu.ravel() + 1.96 * std.ravel(), 
                     mu.ravel() - 1.96 * std.ravel(), 
                     alpha=0.1) 
    #plt.plot(X, Y, 'y--', lw=1, label='Noise-free objective')
    plt.plot(X, mu, 'b-', lw=1, label='Mean function')
    plt.plot(X_sample, Y_sample, 'kx', mew=1, label=noisy_label+' samples')
    if X_next:
        plt.axvline(x=X_next, ls='--', c='k', lw=1)
    if show_legend:
        plt.legend()

def plot_approximation(gpr, X, X_sample, Y_sample, X_next=None, show_legend=False):
    mu, std = gpr.predict(X)
    plt.fill_between(X.ravel(), 
                     mu.ravel() + 1.96 * std, 
                     mu.ravel() - 1.96 * std, 
                     alpha=0.1) 
    #plt.plot(X, Y, 'y--', lw=1, label='Noise-free objective')
    plt.plot(X, mu, 'b-', lw=1, label='Surrogate function')
    plt.plot(X_sample, Y_sample, 'kx', mew=1, label='Noisy samples')
    if X_next:
        plt.axvline(x=X_next, ls='--', c='k', lw=1)
    if show_legend:
        plt.legend()
'''
def plot_acquisition_2d(X, Y, X_next, show_legend=False):
    plt.plot(X, Y, 'r-', lw=1, label='Acquisition function')
    plt.axvline(x=X_next, ls='--', c='k', lw=1, label='Next sampling location')
    if show_legend:
        plt.legend()   '''
def plot_acquisition_2d(X, plot_means, X_next, show_legend=False):
    #print('x',X[0].shape)
    #print('mean',plot_means.shape)
    
    X,Y=grid=np.meshgrid(X[:, 0],X[:, 1])
    x_test=np.vstack([X.flatten(),Y.flatten()]).T
    #plt.contour(X, Y, plot_means.reshape(X.shape[0], X.shape[0]),levels=700, linewidths=0.5, colors='k')
    plt.contourf(X, Y, plot_means.reshape(X.shape[0], X.shape[0]), levels=700, cmap="RdBu_r")
    #plt.clabel(CS, inline=1, fontsize=10)
    #print(X_next)
    plt.plot(X_next.T[:, 0], X_next.T[:, 1], 'co', mew=3, label='Acquisition Fn - Next Sample')
    cntr1 = plt.contourf(X, Y, plot_means.reshape(X.shape[0], X.shape[0]), levels=10, cmap="RdBu_r")
    if show_legend:
        plt.colorbar(cntr1)
        plt.legend()
        
def plot_acquisition(X, Y, X_next, show_legend=False):
    plt.plot(X, Y, 'r-', lw=1, label='Acquisition function')
    plt.axvline(x=X_next, ls='--', c='k', lw=1, label='Next sampling location')
    if show_legend:
        plt.legend()    
        
def plot_convergence(X_sample, Y_sample, n_init=2):
    plt.figure(figsize=(12, 3))

    x = X_sample[n_init:].ravel()
    y = Y_sample[n_init:].ravel()
    r = range(1, len(x)+1)
    
    x_neighbor_dist = [np.abs(a-b) for a, b in zip(x, x[1:])]
    y_max_watermark = np.maximum.accumulate(y)
    
    plt.subplot(1, 2, 1)
    plt.plot(r[1:], x_neighbor_dist, 'bo-')
    plt.xlabel('Iteration')
    plt.ylabel('Distance')
    plt.title('Distance between consecutive x\'s')

    plt.subplot(1, 2, 2)
    plt.plot(r, y_max_watermark, 'ro-')
    plt.xlabel('Iteration')
    plt.ylabel('Best Y')
    plt.title('Value of best selected sample')

In [3]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import ConstantKernel, Matern
import GPy
'''
    Bayesian Optimization Driver Methods
    Kernel, GPR posterios update
'''
def bayOptEI(n_iter, X, X_init, Y_init, f, kernel, mf,\
             mpi=False, lr=0.4,opt=False,bounds=np.array([[0.1, 1]]),plotPlots=False):
    X_sample = X_init
    Y_sample = Y_init
    
    for i in range(n_iter):
        # Update Gaussian process with existing samples
        if(mf):
            gpr = GPy.models.GPRegression(X_sample,Y_sample,kernel=kernel, mean_function=mf)
        else:
            gpr = GPy.models.GPRegression(X_sample,Y_sample,kernel=kernel)
        

        # Obtain next sampling point from the acquisition function (expected_improvement)
        X_next = propose_location(expected_improvement, X_sample, Y_sample, gpr, bounds)[0, 0]
    
        # Obtain next noisy sample from the objective function
        #Y_next = Deep_cont(testPolicy, max_steps)
        #Y_next = f(testPolicy, agentTrainer, lr=X[])
        Y_next = f(gamma=X_next, live=False, lr=lr, best_samples=Y_sample.flatten())
        
        if(plotPlots):
            # Plot samples, surrogate function, noise-free objective and next sampling location
            if(i%2 == 0):
            #if(True):
                plt.subplot(n_iter, 2, i + 1)
                #plt.subplot(n_iter, 2, i+1)
                plot_app(gpr, X, X_sample, Y_sample, X_next, noisy_label='Gamma', show_legend=True)
                plt.title(f'Iteration {i+1}')

                plt.subplot(n_iter, 2, i + 2)
                #plt.subplot(n_iter, 2, i+2)
                plot_acquisition(X, expected_improvement(X, X_sample, Y_sample, gpr, mpi=mpi), X_next, show_legend=True)
    
        # Add sample to previous samples
        X_sample = np.vstack((X_sample, X_next))
        Y_sample = np.vstack((Y_sample, Y_next))
    #plt.tight_layout()
    #gpr.save_model('gprmodel1D', compress=True, save_data=True)
    return X_sample, Y_sample

def bayOptEI_2d(n_iter, X, X_init, Y_init, f, kernel, mpi=False,\
             mf=False, lr=0.4,bounds =np.array([[0.1, 0.9], [0.01, 0.2]]), plotPlots=False):
    noise = 0.
    X_sample = X_init
    Y_sample = Y_init

    for i in range(n_iter):
        if(mf):
            gpr = GPy.models.GPRegression(X_sample,Y_sample,kernel=kernel, mean_function=mf)
        else:
            gpr = GPy.models.GPRegression(X_sample,Y_sample,kernel=kernel)
            
        # Obtain next sampling point from the acquisition function (expected_improvement)
        X_next = propose_location_2d(expected_improvement_2d, X_sample, Y_sample, gpr, bounds)
        # Obtain next noisy sample from the objective function
        #Y_next = Deep_cont(testPolicy, max_steps)
        #Y_next = f(testPolicy, agentTrainer, lr=X[])
        print(X_next)
        Y_next = f(gamma=X_next[0][0], live=False, lr=X_next[1][0], best_samples=Y_sample.flatten())
        if(plotPlots):
            # Plot samples, surrogate function, noise-free objective and next sampling location
            if(i%3 == 0):
                #plt.subplot(n_iter, 2, 2 * i + 1)
                #plt.subplot(n_iter, 2, i+1)
                plt.subplot(n_iter, 3, i+1)
                plot_contour(gpr, (X[:, 0], X[:, 1]), X_sample, Y_sample, X_next, \
                             noisy_label='Gamma', show_legend=i==0)
                plt.title(f'Mean Iteration {i+1}')
                
                plt.subplot(n_iter, 3, i + 2)
                plot_contour_var(gpr, (X[:, 0], X[:, 1]), X_sample, Y_sample, X_next, \
                             noisy_label='Gamma', showLegend=i==0)
                plt.title(f'Variance Iteration {i+1}')
                
                plt.subplot(n_iter, 3, i + 3)
                plot_acquisition_2d(X, expected_improvement_2d(X, X_sample, Y_sample, gpr, mpi=mpi,plot=True), \
                                    X_next, show_legend=i==0)
                plt.title(f'Acquisition {i+1}')
                #plot_acquisition(X, expected_improvement(X, X_sample, Y_sample, gpr), X_next, show_legend=i==0)
    
        # Add sample to previous samples
        X_sample = np.vstack((X_sample, X_next.T))
        Y_sample = np.vstack((Y_sample, Y_next))
    #plt.tight_layout()
    return X_sample, Y_sample